Construct Graph Function

In [8]:
from graph_no_fuzz import Node

def adjust_session(session, frequencies = []):
    if len(session) == 2:
        if session[0] == session[1]:
            session = [session[0]]
            frequencies = [2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 3:
        if session[0] == session[1] and session[0] == session[2]:
            session = session[:1]
            frequencies = [3]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 4:
        if session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1]]
            frequencies = [2, 2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 5:
        if session[0] == session[1] and session[0] == session[2]:
            session = [session[0], session[3], session[4]]
            frequencies = [3, 1, 1]
        elif session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1], session[4]]
            frequencies = [2, 2, 1]
        else:
            frequencies = [1] * len(session)
    else:
        frequencies = [1] * len(session)

    return session, frequencies


def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        # change this to frequency array of length program
        # and add frequency based on index to it    
        freq_addition = []
        session, freq_addition = adjust_session(session=session)

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=freq_addition[0])
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += freq_addition[0]

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = freq_addition[0]
        else:
            cur_node.commands[first_cmd] += freq_addition[0]

        for cmd in range(1, len(session)):

            if cmd >= len(freq_addition):
                    print(session, len(session))
                    print(freq_addition, len(freq_addition))

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=freq_addition[cmd])
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += freq_addition[cmd]
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = freq_addition[cmd]
            else:
                child_node.commands[session[cmd]] += freq_addition[cmd]
            
            cur_node = child_node
        
    return command_dict

Get Prediction Function

In [9]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    # previous command should be command_list[depth-1]
    return node.get_prediction(previous_command=command_list[graph_depth-1], num_to_return=result_size), frequencies

In [17]:
from thefuzz import fuzz
from thefuzz import process

from statistics import mean

from tqdm import tqdm

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy_colearn(train_data, test_data, fail_list = [], fail_frequencies= [], succeed_list = [], succeed_frequencies = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=-1):
    train_graph = construct_graph(train_data)
    
    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    if (graph_depth != -1):
        test_data = [x for x in test_data if len(x) > graph_depth]
    else:
        test_data = [x for x in test_data if len(x) >= 2]

    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in tqdm(test_data, leave=False):
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct
        prev5_correct =  correct_with_5

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    succeed_frequencies.append(mean(frequencies))

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                    break

            if prev_correct == correct:
                fail_frequencies.append(mean(frequencies))
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100 and prev_correct == correct:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100 and prev5_correct == correct_with_5:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                              
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.2f} |\n Average Fail Freq: {:.2f}'.format(100 * correct/(test_size-none_count), 100 * correct_with_5/(test_size-none_count), 100 * correct_with_15/(test_size-none_count), 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies)/len(train_data), mean(fail_frequencies)/len(train_data))


In [11]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:06<00:00,  8.83it/s]


Colearning train tests, all but last file

In [12]:
science_session_train = parser.parse_commands_per_session(parser.scientists_files[:-1])
science_session_train = parser.parse_commands_into_subsets_sliding_window(science_session_train, 5)
science_session_train = parser.replace_args_nested(science_session_train)

experienced_session_train = parser.parse_commands_per_session(parser.experienced_files[:-1])
experienced_session_train = parser.parse_commands_into_subsets_sliding_window(experienced_session_train, 5)
experienced_session_train = parser.replace_args_nested(experienced_session_train)

non_session_train = parser.parse_commands_per_session(parser.non_programmers_files[:-1])
non_session_train = parser.parse_commands_into_subsets_sliding_window(non_session_train, 5)
non_session_train = parser.replace_args_nested(non_session_train)

novice_session_train = parser.parse_commands_per_session(parser.novice_files[:-1])
novice_session_train = parser.parse_commands_into_subsets_sliding_window(novice_session_train, 5)
novice_session_train = parser.replace_args_nested(novice_session_train)

100%|██████████| 55/55 [00:06<00:00,  8.30it/s]


Colearning test set, last file

In [13]:
science_session_test = parser.parse_commands_per_session(parser.scientists_files[-1:])
science_session_test = parser.parse_commands_into_subsets_sliding_window(science_session_test, 5)
science_session_test = parser.replace_args_nested(science_session_test)

experienced_session_test = parser.parse_commands_per_session(parser.experienced_files[-1:])
experienced_session_test = parser.parse_commands_into_subsets_sliding_window(experienced_session_test, 5)
experienced_session_test = parser.replace_args_nested(experienced_session_test)

non_session_test = parser.parse_commands_per_session(parser.non_programmers_files[-1:])
non_session_test = parser.parse_commands_into_subsets_sliding_window(non_session_test, 5)
non_session_test = parser.replace_args_nested(non_session_test)

novice_session_test = parser.parse_commands_per_session(parser.novice_files[-1:])
novice_session_test = parser.parse_commands_into_subsets_sliding_window(novice_session_test, 5)
novice_session_test = parser.replace_args_nested(novice_session_test)

100%|██████████| 1/1 [00:00<00:00,  9.80it/s]


Accuracy predicting N+1 command from N commands with all commands in graph

TODO: rename/add train/test args

In [20]:
all_train = science_session_train + experienced_session_train + non_session_train + novice_session_train
all_test = science_session_test + experienced_session_test + non_session_test + novice_session_test

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy_colearn(science_session_train, science_session_test, graph_depth=i))
    print("Experienced session:\n", get_accuracy_colearn(experienced_session_train, experienced_session_test, graph_depth=i))
    print("Non programmer session:\n", get_accuracy_colearn(non_session_train, non_session_test, graph_depth=i))
    print("Novice session:\n", get_accuracy_colearn(novice_session_train, novice_session_test, graph_depth=i))
    print("All sessions:\n", get_accuracy_colearn(all_train, all_test, graph_depth=i))

i=-1
print("Accuracy predicting command N from commands 1 through N-1:\n")
print("Scientist session:\n", get_accuracy_colearn(science_session_train, science_session_test, graph_depth=i))
print("Experienced session:\n", get_accuracy_colearn(experienced_session_train, experienced_session_test, graph_depth=i))
print("Non programmer session:\n", get_accuracy_colearn(non_session_train, non_session_test, graph_depth=i))
print("Novice session:\n", get_accuracy_colearn(novice_session_train, novice_session_test, graph_depth=i))
print("All sessions:\n", get_accuracy_colearn(all_train, all_test, graph_depth=i))

Accuracy predicting command 2 from commands 1 through 1:



Scientist session:
 Correct Proportion: 35.14% |
 Correct in 5 not 3: 40.94% |
 Correct in 15 not 5: 53.86% |
 Has Prediction and is Correct: 35.14% |
 Incorrect Proportion: 64.45% |
 None Proportion: 0.62% |
 First Prediction: 20.50% |
 Average Success Freq: 0.28 |
 Average Fail Freq: 0.19


Experienced session:
 Correct Proportion: 28.65% |
 Correct in 5 not 3: 32.78% |
 Correct in 15 not 5: 43.98% |
 Has Prediction and is Correct: 28.65% |
 Incorrect Proportion: 62.79% |
 None Proportion: 12.00% |
 First Prediction: 14.16% |
 Average Success Freq: 0.54 |
 Average Fail Freq: 0.36


Non programmer session:
 Correct Proportion: 22.79% |
 Correct in 5 not 3: 33.61% |
 Correct in 15 not 5: 51.57% |
 Has Prediction and is Correct: 22.79% |
 Incorrect Proportion: 76.62% |
 None Proportion: 0.76% |
 First Prediction: 15.62% |
 Average Success Freq: 1.87 |
 Average Fail Freq: 1.25


Novice session:
 Correct Proportion: 40.66% |
 Correct in 5 not 3: 42.68% |
 Correct in 15 not 5: 44.14% |
 Has Prediction and is Correct: 40.66% |
 Incorrect Proportion: 57.72% |
 None Proportion: 2.74% |
 First Prediction: 6.33% |
 Average Success Freq: 0.54 |
 Average Fail Freq: 0.38


...

Scientist session:
 Correct Proportion: 31.32% |
 Correct in 5 not 3: 37.54% |
 Correct in 15 not 5: 46.82% |
 Has Prediction and is Correct: 31.32% |
 Incorrect Proportion: 58.13% |
 None Proportion: 15.36% |
 First Prediction: 16.79% |
 Average Success Freq: 0.45 |
 Average Fail Freq: 0.33


Experienced session:
 Correct Proportion: 29.35% |
 Correct in 5 not 3: 33.58% |
 Correct in 15 not 5: 41.66% |
 Has Prediction and is Correct: 29.35% |
 Incorrect Proportion: 40.41% |
 None Proportion: 42.81% |
 First Prediction: 10.70% |
 Average Success Freq: 0.79 |
 Average Fail Freq: 0.59


Non programmer session:
 Correct Proportion: 25.09% |
 Correct in 5 not 3: 31.85% |
 Correct in 15 not 5: 44.85% |
 Has Prediction and is Correct: 25.09% |
 Incorrect Proportion: 62.80% |
 None Proportion: 16.16% |
 First Prediction: 11.82% |
 Average Success Freq: 2.62 |
 Average Fail Freq: 1.95


Novice session:
 Correct Proportion: 32.98% |
 Correct in 5 not 3: 37.85% |
 Correct in 15 not 5: 46.96% |
 Has Prediction and is Correct: 32.98% |
 Incorrect Proportion: 42.44% |
 None Proportion: 36.67% |
 First Prediction: 8.16% |
 Average Success Freq: 0.78 |
 Average Fail Freq: 0.59


All sessions:
 Correct Proportion: 29.37% |
 Correct in 5 not 3: 34.75% |
 Correct in 15 not 5: 44.64% |
 Has Prediction and is Correct: 29.37% |
 Incorrect Proportion: 48.69% |
 None Proportion: 31.06% |
 First Prediction: 11.38% |
 Average Success Freq: 0.22 |
 Average Fail Freq: 0.16
